In [1]:
import os

In [9]:
def get_size(start_path = '.'):
    total_size = 0
    number_of_files = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
                number_of_files += 1

    return total_size, number_of_files

In [10]:
x, num = get_size('../../dataset/Reddit_Provenance_Datasets/data/')

In [11]:
x=x/(1024**3)

In [12]:
x

5.253659635782242

In [13]:
num

10317

### Setting up eva

In [17]:
from IPython.core.magic import register_cell_magic

from eva.server.db_api import connect
import nest_asyncio

nest_asyncio.apply()
cur = connect(host="127.0.0.1", port=5432).cursor()

@register_cell_magic
def evaql(line, cell):
    x = cell.split(';')
    for i in x[:-1]:
        cur.execute(i)
        res = cur.fetch_all()
        print(res)

### Load images into table;

In [ ]:
start_path= "/nethome/kravicha3/aryan/project/dataset/Reddit_Provenance_Datasets/data/"
for dirpath, dirnames, filenames in os.walk(start_path):
    for f in filenames:
        fp = os.path.join(dirpath, f)
        if not os.path.islink(fp):
            cur.execute(f'LOAD IMAGE "{fp}" INTO reddit;')
            res = cur.fetch_all()
            print(res)

In [ ]:
cur.execute("SELECT * FROM reddit;")
res = cur.fetch_all()

In [ ]:
res

#### Getting sqlite connect  

    CREATE TABLE sqlite_schema(
      type text,
      name text,
      tbl_name text,
      rootpage integer,
      sql text
    );

In [8]:
import sqlite3
con = sqlite3.connect('/nethome/kravicha3/.eva/0.1.5+dev/eva_catalog.db')

In [9]:
c = con.cursor()

In [10]:
c.execute("SELECT * FROM sqlite_master;")
res = c.fetchall()
for i in res:
    print(i[:-1], end="\n\n")

('table', 'table_catalog', 'table_catalog', 2)

('index', 'sqlite_autoindex_table_catalog_1', 'table_catalog', 3)

('table', 'udf_catalog', 'udf_catalog', 4)

('index', 'sqlite_autoindex_udf_catalog_1', 'udf_catalog', 5)

('table', 'column_catalog', 'column_catalog', 6)

('index', 'sqlite_autoindex_column_catalog_1', 'column_catalog', 7)

('table', 'udfio_catalog', 'udfio_catalog', 8)

('index', 'sqlite_autoindex_udfio_catalog_1', 'udfio_catalog', 9)

('table', 'index_catalog', 'index_catalog', 10)

('index', 'sqlite_autoindex_index_catalog_1', 'index_catalog', 11)

('table', '192111ccbbbfc5042415841dfaa9f90a', '192111ccbbbfc5042415841dfaa9f90a', 12)



In [15]:
c.execute("SELECT * FROM '192111ccbbbfc5042415841dfaa9f90a' LIMIT 1;")
res = c.fetchall()
print(res)

[(1, '/nethome/kravicha3/aryan/project/dataset/Reddit_Provenance_Datasets/data/_This_cat_plotting_to_kill_someone/g1327_czcqbl6.jpg')]


In [13]:
c.execute("SELECT COUNT(*) FROM '192111ccbbbfc5042415841dfaa9f90a';")
res = c.fetchall()
print(res)

[(9801,)]


## Creating index

In [7]:
from IPython.core.magic import register_cell_magic

from eva.server.db_api import connect
import nest_asyncio

nest_asyncio.apply()
cur = connect(host="127.0.0.1", port=5432).cursor()

@register_cell_magic
def evaql(line, cell):
    x = cell.split(';')
    for i in x[:-1]:
        cur.execute(i)
        res = cur.fetch_all()
        print(res)

In [9]:
# %%evaql
# CREATE UDF FeatureExtractor
# INPUT  (frame NDARRAY UINT8(3, ANYDIM, ANYDIM))
# OUTPUT (features NDARRAY FLOAT32(2, 1024))
# TYPE  Classification
# IMPL  './eva/udfs/feature_extractor.py';

In [ ]:
# %%evaql
# CREATE INDEX dataindex ON reddit (FeatureExtractor(data)) USING HNSW;

In [ ]:
# %%evaql
# SELECT * FROM reddit;

### Seeing if index creation is complete
no. of images = 10317

In [2]:
INDEX_HOME = "/nethome/kravicha3/.eva/0.1.5+dev/index/"

In [3]:
import faiss

In [5]:
index = faiss.read_index(INDEX_HOME+"HNSW_dataindex.index")

In [6]:
index.ntotal

9801

### Checking Similarity search speed

running query on eva_client

ans: similarity search takes about 250 seconds per image.

In [4]:
from IPython.core.magic import register_cell_magic

from eva.server.db_api import connect
import nest_asyncio

nest_asyncio.apply()
cur = connect(host="127.0.0.1", port=5432).cursor()

@register_cell_magic
def evaql(line, cell):
    x = cell.split(';')
    for i in x[:-1]:
        cur.execute(i)
        res = cur.fetch_all()
        print(res)

In [5]:
%%evaql
SELECT * FROM reddit ORDER BY Similarity(FeatureExtractor(Open("/nethome/kravicha3/aryan/project/dataset/Reddit_Provenance_Datasets/data/_This_cat_plotting_to_kill_someone/g1327_czcqbl6.jpg")), FeatureExtractor(data)) LIMIT 5;

@status: ResponseStatus.FAIL
@batch: 
 None
@error: None
